In [2]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [1]:
!aws s3 ls s3://jfrechmsml650output/csv/

2024-11-09 16:28:12          0 
2024-11-09 16:35:49         45 cctv052x2004080516x01638_test.csv
2024-11-13 02:30:50         45 cctv052x2004080516x01639_test.csv
2024-11-13 02:30:59         45 cctv052x2004080516x01640_test.csv
2024-11-13 02:31:12         43 cctv052x2004080516x01641_test.csv
2024-11-13 02:31:23         45 cctv052x2004080516x01642_test.csv
2024-11-13 02:31:34         44 cctv052x2004080516x01643_test.csv
2024-11-13 02:31:44         45 cctv052x2004080516x01644_test.csv
2024-11-13 02:31:54         45 cctv052x2004080516x01645_test.csv
2024-11-13 02:32:05         45 cctv052x2004080516x01646_test.csv
2024-11-13 02:32:17         45 cctv052x2004080516x01647_test.csv
2024-11-13 02:32:27         45 cctv052x2004080516x01648_test.csv
2024-11-13 02:32:39         44 cctv052x2004080516x01649_test.csv
2024-11-13 02:32:50         45 cctv052x2004080516x01650_test.csv
2024-11-13 02:33:02         44 cctv052x2004080517x01652_test.csv
2024-11-13 02:33:14         43 cctv052x2004080517x01653_te

In [9]:
import boto3
from botocore.exceptions import ClientError

# Create an S3 client
s3 = boto3.client('s3')

# Define the bucket and folder path
bucket_name = 'jfrechmsml650output'
folder_path = 'csv/'

# Try to list objects in the specified folder
try:
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_path)
    
    # Check if any objects were returned
    if 'Contents' in response:
        print(f"Objects found in {bucket_name}/{folder_path}:")
        for obj in response['Contents']:
            print(obj['Key'])
    else:
        print(f"No objects found in {bucket_name}/{folder_path}.")
except ClientError as e:
    # Catch and display any errors that occur
    print(f"Error: {e}")

Objects found in jfrechmsml650output/csv/:
csv/
csv/cctv052x2004080516x01638_test.csv
csv/cctv052x2004080516x01639_test.csv
csv/cctv052x2004080516x01640_test.csv
csv/cctv052x2004080516x01641_test.csv
csv/cctv052x2004080516x01642_test.csv
csv/cctv052x2004080516x01643_test.csv
csv/cctv052x2004080516x01644_test.csv
csv/cctv052x2004080516x01645_test.csv
csv/cctv052x2004080516x01646_test.csv
csv/cctv052x2004080516x01647_test.csv
csv/cctv052x2004080516x01648_test.csv
csv/cctv052x2004080516x01649_test.csv
csv/cctv052x2004080516x01650_test.csv
csv/cctv052x2004080517x01652_test.csv
csv/cctv052x2004080517x01653_test.csv
csv/cctv052x2004080517x01654_test.csv
csv/cctv052x2004080517x01655_test.csv
csv/cctv052x2004080517x01656_test.csv
csv/cctv052x2004080517x01657_test.csv
csv/cctv052x2004080517x01658_test.csv
csv/cctv052x2004080517x01659_test.csv
csv/cctv052x2004080517x01660_test.csv
csv/cctv052x2004080517x01661_test.csv
csv/cctv052x2004080517x01662_test.csv
csv/cctv052x2004080517x01663_test.csv
cs

In [3]:
from pandas import DataFrame, read_csv, concat
import boto3
from os import remove

# Initialize S3 client
s3 = boto3.client('s3')

# Choose input and output buckets
output_bucket_name = 'jfrechmsml650output'
concatenated_csv = 'concatenated_output.csv'

# List files already in the output directory
written_output_files = s3.list_objects(Bucket=output_bucket_name, Prefix='csv/', Delimiter='/')
written_output_files = [i['Key'] for i in written_output_files.get('Contents', [])]

# Filter out empty keys or directories
csv_files = [key for key in written_output_files if key.endswith('.csv')]

# Initialize an empty list to store DataFrames
dataframes = []

# Process each CSV file
for csv_file in csv_files:
    local_filename = csv_file.split('/')[-1]  # Extract the file name
    # Download the file locally
    s3.download_file(output_bucket_name, csv_file, local_filename)
    
    # Load the CSV into a DataFrame
    df = read_csv(local_filename)
    dataframes.append(df)
    
    # Clean up the local file
    remove(local_filename)

# Concatenate all DataFrames
concat_df = concat(dataframes, ignore_index=True)

# # Save the concatenated DataFrame to a new CSV file
# final_df.to_csv(concatenated_csv, index=False)

# # Optional: Upload the concatenated CSV back to S3
# s3.upload_file(concatenated_csv, output_bucket_name, f'csv/{concatenated_csv}')

# # Remove the local concatenated file
# remove(concatenated_csv)

# print(f"Concatenated CSV saved to {concatenated_csv} and uploaded to bucket.")

concat_df.head()

,n_cars,traffic_speed
0,18.0,13.292431
1,19.0,10.643991
2,20.0,14.672661
3,17.0,7.979585
4,17.0,10.490092


In [4]:
# Data types
# Camera ID, Date-Time (Y-M-D-H-M) Weekend WeekofMonth Season Speed (traffic_speed) Density (n_cars)
concat_df['date_time'] = pd.to_datetime('2024-11-18 15:45:00')
concat_df['camera_id'] = '0'
concat_df

,n_cars,traffic_speed,date_time,camera_id
0,18.0,13.292431,2024-11-18 15:45:00,0
1,19.0,10.643991,2024-11-18 15:45:00,0
2,20.0,14.672661,2024-11-18 15:45:00,0
3,17.0,7.979585,2024-11-18 15:45:00,0
4,17.0,10.490092,2024-11-18 15:45:00,0
...,...,...,...,...
125,9.0,-1.000000,2024-11-18 15:45:00,0
126,3.0,-1.000000,2024-11-18 15:45:00,0
127,4.0,16.561155,2024-11-18 15:45:00,0
128,2.0,13.727386,2024-11-18 15:45:00,0


In [16]:
from datetime import datetime

# Take the datetime object and turn it into the other needed columns
def extract_date_info(dt):
    # Extract basic components
    year = dt.year
    month = dt.month
    day = dt.day
    hour = dt.hour
    minute = dt.minute
    weekday = dt.weekday()  # 0=Monday, 6=Sunday
    is_weekend = weekday >= 5

    # Calculate the week of the month
    first_day_of_month = dt.replace(day=1)
    week_of_month = (dt.day + first_day_of_month.weekday()) // 7 + 1

    # Determine the season (Northern Hemisphere)
    if month in (12, 1, 2):
        season = "Winter"
    elif month in (3, 4, 5):
        season = "Spring"
    elif month in (6, 7, 8):
        season = "Summer"
    else:
        season = "Fall"

    return year, month, day, hour, minute, is_weekend, week_of_month, season

# Example usage with a datetime object
date_time_obj = datetime(2024, 11, 18, 15, 45, 0)  # Example datetime object
info = extract_date_info(date_time_obj)
print(info)

(2024, 11, 18, 15, 45, False, 4, 'Fall')


In [17]:
# DF for cols
traffic_df = pd.DataFrame(columns=['camera_id', 'speed', 'density', 'year', 'month', 'day', 'hour', 'minute', 'weekend', 'week_of_month', 'season'])
traffic_df

,camera_id,speed,density,year,month,day,hour,minute,weekend,week_of_month,season


In [18]:
concat_df[['year', 'month', 'day', 'hour', 'minute', 'is_weekend', 'week_of_month', 'season']] = concat_df['date_time'].apply(lambda x: pd.Series(extract_date_info(x)))
concat_df

,n_cars,traffic_speed,date_time,camera_id,year,month,day,hour,minute,is_weekend,week_of_month,season
0,18.0,13.292431,2024-11-18 15:45:00,0,2024,11,18,15,45,False,4,Fall
1,19.0,10.643991,2024-11-18 15:45:00,0,2024,11,18,15,45,False,4,Fall
2,20.0,14.672661,2024-11-18 15:45:00,0,2024,11,18,15,45,False,4,Fall
3,17.0,7.979585,2024-11-18 15:45:00,0,2024,11,18,15,45,False,4,Fall
4,17.0,10.490092,2024-11-18 15:45:00,0,2024,11,18,15,45,False,4,Fall
...,...,...,...,...,...,...,...,...,...,...,...,...
125,9.0,-1.000000,2024-11-18 15:45:00,0,2024,11,18,15,45,False,4,Fall
126,3.0,-1.000000,2024-11-18 15:45:00,0,2024,11,18,15,45,False,4,Fall
127,4.0,16.561155,2024-11-18 15:45:00,0,2024,11,18,15,45,False,4,Fall
128,2.0,13.727386,2024-11-18 15:45:00,0,2024,11,18,15,45,False,4,Fall


In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

def preprocess_expanded_df(data):
    """
    Preprocess the expanded DataFrame for use with the AWS Random Cut Forest (RCF) model.
    
    This function:
    - Removes `date_time`.
    - Encodes and transforms categorical and cyclical features.
    - Normalizes numerical features.

    Parameters:
    - data (pd.DataFrame): Expanded DataFrame with columns:
      ['n_cars', 'traffic_speed', 'date_time', 'camera_id', 
       'year', 'month', 'day', 'hour', 'minute', 
       'is_weekend', 'week_of_month', 'season'].

    Returns:
    - pd.DataFrame: Processed DataFrame ready for RCF.
    """
    # Drop the `date_time` column
    data = data.drop(columns=["date_time"])
    
    # Encode `is_weekend` as integer (0 or 1)
    data["is_weekend"] = data["is_weekend"].astype(int)
    
    # Map `season` to numeric values if it's of type object
    if data["season"].dtype == "object":
        season_mapping = {"Winter": 0, "Spring": 1, "Summer": 2, "Fall": 3}
        data["season"] = data["season"].map(season_mapping)
        if data["season"].isna().any():
            raise ValueError("Unmapped season value detected in the data.")
    
    # Define a function for cyclical transformations
    def add_cyclical_features(data, column, max_value):
        data[f"{column}_sin"] = np.sin(2 * np.pi * data[column] / max_value)
        data[f"{column}_cos"] = np.cos(2 * np.pi * data[column] / max_value)
        return data
    
    # Apply cyclical transformations
    data = add_cyclical_features(data, "month", 12)          # Cyclical for months
    data = add_cyclical_features(data, "day", 31)            # Cyclical for days
    data = add_cyclical_features(data, "hour", 24)           # Cyclical for hours
    data = add_cyclical_features(data, "minute", 60)         # Cyclical for minutes
    data = add_cyclical_features(data, "week_of_month", 5)   # Cyclical for weeks of the month
    data = add_cyclical_features(data, "season", 4)          # Cyclical for seasons
    data = add_cyclical_features(data, "year", 10)           # Cyclical for years (assuming a 10-year cycle, adjust as needed)
    
    # Drop the original cyclical columns
    columns_to_drop = ["month", "day", "hour", "minute", "week_of_month", "season", "year"]
    data = data.drop(columns=columns_to_drop)
    
    # Normalize numerical features
    numerical_columns = [
        "n_cars", "traffic_speed", "camera_id", "is_weekend"
    ]
    scaler = StandardScaler()
    data[numerical_columns] = scaler.fit_transform(data[numerical_columns])
    
    return data

In [13]:
concat_df.dtypes

n_cars                  float64
traffic_speed           float64
date_time        datetime64[ns]
camera_id                object
year                      int64
month                     int64
day                       int64
hour                      int64
minute                    int64
is_weekend                 bool
week_of_month             int64
season                   object
dtype: object

In [22]:
preprocessed_df = preprocess_expanded_df(concat_df)
preprocessed_df

,n_cars,traffic_speed,camera_id,is_weekend,month_sin,month_cos,day_sin,day_cos,hour_sin,hour_cos,minute_sin,minute_cos,week_of_month_sin,week_of_month_cos,season_sin,season_cos,year_sin,year_cos
0,1.138206,0.259712,0.0,0.0,-0.5,0.866025,-0.485302,-0.874347,-0.707107,-0.707107,-1.0,-1.836970e-16,-0.951057,0.309017,-1.0,-1.836970e-16,0.587785,-0.809017
1,1.328394,-0.041133,0.0,0.0,-0.5,0.866025,-0.485302,-0.874347,-0.707107,-0.707107,-1.0,-1.836970e-16,-0.951057,0.309017,-1.0,-1.836970e-16,0.587785,-0.809017
2,1.518583,0.416497,0.0,0.0,-0.5,0.866025,-0.485302,-0.874347,-0.707107,-0.707107,-1.0,-1.836970e-16,-0.951057,0.309017,-1.0,-1.836970e-16,0.587785,-0.809017
3,0.948017,-0.343792,0.0,0.0,-0.5,0.866025,-0.485302,-0.874347,-0.707107,-0.707107,-1.0,-1.836970e-16,-0.951057,0.309017,-1.0,-1.836970e-16,0.587785,-0.809017
4,0.948017,-0.058615,0.0,0.0,-0.5,0.866025,-0.485302,-0.874347,-0.707107,-0.707107,-1.0,-1.836970e-16,-0.951057,0.309017,-1.0,-1.836970e-16,0.587785,-0.809017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,-0.573492,-1.363814,0.0,0.0,-0.5,0.866025,-0.485302,-0.874347,-0.707107,-0.707107,-1.0,-1.836970e-16,-0.951057,0.309017,-1.0,-1.836970e-16,0.587785,-0.809017
126,-1.714623,-1.363814,0.0,0.0,-0.5,0.866025,-0.485302,-0.874347,-0.707107,-0.707107,-1.0,-1.836970e-16,-0.951057,0.309017,-1.0,-1.836970e-16,0.587785,-0.809017
127,-1.524435,0.631018,0.0,0.0,-0.5,0.866025,-0.485302,-0.874347,-0.707107,-0.707107,-1.0,-1.836970e-16,-0.951057,0.309017,-1.0,-1.836970e-16,0.587785,-0.809017
128,-1.904812,0.309120,0.0,0.0,-0.5,0.866025,-0.485302,-0.874347,-0.707107,-0.707107,-1.0,-1.836970e-16,-0.951057,0.309017,-1.0,-1.836970e-16,0.587785,-0.809017
